<a href="https://colab.research.google.com/github/raqueeb/nlp_bangla/blob/master/Chap_7_transformers_bert_Bangla_extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## হাতেকলমে ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং: শুরুর ধারণা

[৭ম অধ্যায়](https://aiwithr.github.io/nlpbook/)

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "raqueeb" # username from the json file
os.environ['KAGGLE_KEY'] = "7d6bb294264708d37ce5520a6fca7252"

In [2]:
!kaggle datasets files furcifer/bangla-newspaper-dataset

name                  size  creationDate         
--------------------  ----  -------------------  
data_v2/data_v2.json   2GB  2020-10-21 06:09:48  
data/data.json         4GB  2020-10-21 06:09:48  


In [3]:
!kaggle datasets download furcifer/bangla-newspaper-dataset -f data_v2/data_v2.json

 95% 422M/444M [00:01<00:00, 269MB/s]
100% 444M/444M [00:01<00:00, 255MB/s]


In [4]:
!unzip data_v2.json.zip

Archive:  data_v2.json.zip
  inflating: data_v2.json            


In [5]:
import json
with open('data_v2.json', encoding='utf-8') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['content'])
    labels.append(item['category'])

In [ ]:
set(labels)

{'bangladesh',
 'economy',
 'education',
 'entertainment',
 'international',
 'life-style',
 'opinion',
 'sports',
 'technology'}

In [ ]:
categories = list(set(labels))

In [ ]:
categories

['life-style',
 'economy',
 'sports',
 'entertainment',
 'bangladesh',
 'education',
 'international',
 'technology',
 'opinion']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  sentences, labels, test_size=0.2, random_state=42)

In [7]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 138kB/s 
     |████████████████████████████████| 983kB 51.1MB/s 
     |████████████████████████████████| 266kB 52.0MB/s 
     |████████████████████████████████| 1.3MB 49.3MB/s 
     |████████████████████████████████| 1.1MB 36.7MB/s 
     |████████████████████████████████| 471kB 53.3MB/s 
     |████████████████████████████████| 890kB 45.7MB/s 
     |████████████████████████████████| 2.9MB 47.5MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.2-cp36-none-any.whl size=25276306 sha256=df93aa67add6abca7bbab22b622e7d43e22fbe8dea4f36301dc326fba7c90b64
  Stored in directory: /root/.cache/pip/wheels/fe/56/00/25444c352cc843e5c5daea0e9517a192878ae22c2c6b5f4573
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993193 sha256=cf892e95d12583d0fdf7e10063e19ba4195169626c6e58fc8c0bd8485b716a92
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d40a60ea06e6d669e8e5012e6c8b947a57
  Created wheel for syn

In [8]:
import ktrain
from ktrain import text

In [9]:
MODEL_NAME = 'sagorsarker/bangla-bert-base'
t = text.Transformer(MODEL_NAME, maxlen=500)

In [10]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 268
	95percentile : 669
	99percentile : 1079


Is Multi-Label? False
preprocessing test...
language: bn
test sequence lengths:
	mean : 269
	95percentile : 669
	99percentile : 1075


In [11]:
model = t.get_classifier()

In [ ]:
# ট্রেনিং করে নিচ্ছি
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=2)

In [ ]:
%matplotlib inline
learner.lr_find(show_plot=True, max_epochs=1)

In [ ]:
# preproc = t
# save model and Preprocessor instance after partially training
# ktrain.get_predictor(model, preproc).save('/tmp/my_predictor')

In [ ]:
# reload Predictor and extract model
# model = ktrain.load_predictor('/tmp/my_predictor').model

In [13]:
learner.fit_onecycle(8e-5, 1)



begin training using onecycle policy with max lr of 8e-05...
  4280/163388 [..............................] - ETA: 14:04:04 - loss: 0.4303 - accuracy: 0.8720

begin training using onecycle policy with max lr of 8e-05...
 56388/163388 [=========>....................] - ETA: 9:27:39 - loss: 0.4966 - accuracy: 0.8542Buffered data was truncated after reaching the output size limit.

In [ ]:
learner.validate(class_names=t.get_classes())

In [ ]:
# the one we got most wrong
learner.view_top_losses(n=1, preproc=t)

In [14]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [15]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
predictor.save('/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor_kt')
# save model using transformers API after partially training
learner.model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor')


In [17]:
!ls -alh '/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor_kt'

total 628M
-rw------- 1 root root  902 Dec 14 11:10 config.json
-rw------- 1 root root 628M Dec 14 11:10 tf_model.h5
-rw------- 1 root root 3.0K Dec 14 11:10 tf_model.preproc


In [18]:
learner.model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  164396544 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  6921      
Total params: 164,403,465
Trainable params: 164,403,465
Non-trainable params: 0
_________________________________________________________________


In [19]:
predictor.predict('প্রথম আলো সম্পাদকের আইনজীবী মোস্তাফিজুর রহমান খান সাংবাদিকদের জানান, গেল ৬ ডিসেম্বর প্রথম আলো সম্পাদকের বিরুদ্ধে মামলাটি বাতিল চেয়ে আবেদন করা হয়েছিল। এতে যুক্তি দেখানো হয়, মামলার তদন্ত প্রতিবেদনে মতিউর রহমানের সম্পৃক্ততার অভিযোগ ছিল না।')

'bangladesh'

In [24]:
predictor.explain('প্রথম আলো সম্পাদকের আইনজীবী মোস্তাফিজুর রহমান খান সাংবাদিকদের জানান, গেল ৬ ডিসেম্বর প্রথম আলো সম্পাদকের বিরুদ্ধে মামলাটি বাতিল চেয়ে আবেদন করা হয়েছিল। এতে যুক্তি দেখানো হয়, মামলার তদন্ত প্রতিবেদনে মতিউর রহমানের সম্পৃক্ততার অভিযোগ ছিল না।')

In [25]:
predictor.predict('ই-কমার্স খাতে বিশেষ অবদান রাখায় এবছর ‘৭ম ডিজিটাল ওয়ার্ল্ড-২০২০’ এ সেরা পাবলিক প্রতিষ্ঠান হিসেবে বাণিজ্য মন্ত্রণালয়কে সম্মাননা ও পুরস্কার প্রদান করা হয়েছে।')

'bangladesh'

In [26]:
predictor.explain('ই-কমার্স খাতে বিশেষ অবদান রাখায় এবছর ‘৭ম ডিজিটাল ওয়ার্ল্ড-২০২০’ এ সেরা পাবলিক প্রতিষ্ঠান হিসেবে বাণিজ্য মন্ত্রণালয়কে সম্মাননা ও পুরস্কার প্রদান করা হয়েছে।')

In [27]:
predictor.predict('বাংলাদেশ টেলিভিশনে গান হয়, সংস্কৃতিকে তুলে ধরতে হবে আমাদের।')

'bangladesh'

In [28]:
predictor.predict_proba('বাংলাদেশে টেলিভিশনে গান হয়, সংস্কৃতিকে তুলে ধরতে হবে আমাদের।')

array([0.61413276, 0.04161949, 0.0186341 , 0.06025607, 0.06673001,
       0.02094929, 0.0295498 , 0.11082707, 0.03730145], dtype=float32)

In [29]:
predictor.predict('বাংলাদেশে ফুটবল খুব বেশি খেলা হয়, ক্রিকেট কম নয়।')

'bangladesh'

In [21]:
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-olugz2re
  Running command git clone -q https://github.com/amaiya/eli5 /tmp/pip-req-build-olugz2re
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfkeras_0_10_1' set up to track remote branch 'tfkeras_0_10_1' from 'origin'.
  Created wheel for eli5: filename=eli5-0.10.1-py2.py3-none-any.whl size=106831 sha256=6cb8a0cddb98797dc45c6166e9ec109ae953198d437616bae5564b38baf3dc75
  Stored in directory: /tmp/pip-ephem-wheel-cache-bc4a84ng/wheels/51/59/0a/0f48442b8d209583a4453580938d7ba2270aca40edacee6d45
Successfully built eli5


In [30]:
predictor.explain('শিক্ষা মন্ত্রণালয়ে এমন প্রস্তাব দিয়েছে যেখানে ছাত্রছাত্রীদেরদের বই খাতা কেনা হবে সবার জন্য।')

In [ ]:
# টেন্সরফ্লোতে মডেল সেভ করা
# model.save('/content/gdrive/My Drive/Colab Notebooks/saved_model/my_model')

In [39]:
# reload the model using transformers directly
from transformers import *
model = TFAutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor')
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [40]:
model.summary()

Model: "tf_bert_for_sequence_classification_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  164396544 
_________________________________________________________________
dropout_150 (Dropout)        multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  6921      
Total params: 164,403,465
Trainable params: 164,403,465
Non-trainable params: 0
_________________________________________________________________


In [36]:
reloaded_predictor = ktrain.load_predictor('/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor_kt')

In [37]:
reloaded_predictor.predict_proba('বঙ্গবন্ধু টি–টোয়েন্টি কাপে কাল জেমকন খুলনার হয়ে খেলার ম্যাচে ৪ ওভারে ২৮ রানে ১ উইকেট নেন মাশরাফি বিন মুর্তজা।')

array([0.61413276, 0.04161949, 0.0186341 , 0.06025607, 0.06673001,
       0.02094929, 0.0295498 , 0.1108271 , 0.03730145], dtype=float32)

In [38]:
reloaded_predictor.get_classes()

['bangladesh',
 'economy',
 'education',
 'entertainment',
 'international',
 'life-style',
 'opinion',
 'sports',
 'technology']

In [33]:
!ls -ahl '/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor'

total 628M
-rw------- 1 root root  902 Dec 14 11:10 config.json
-rw------- 1 root root 628M Dec 14 11:11 tf_model.h5


In [31]:
  predictor.explain('ডিজিটাল বাংলাদেশ গড়ার পরিকল্পনার অংশ হিসাবে ই-কমার্সে দক্ষ জনবল বৃদ্ধি ও কর্মসংস্থান সৃষ্টি করার লক্ষ্যে বাণিজ্য মন্ত্রণালয়ের ‘ই-বাণিজ্য করবো নিজের ব্যবসা গড়বো’ প্রকল্পের আওতায় দেশব্যাপী ই-কমার্স বিষয়ক প্রশিক্ষণ পরিচালিত হচ্ছে। চতুর্থ শিল্প বিপ্লবের চ্যালেঞ্জ মোকাবেলায় ই-কমার্সে দক্ষ জনবল আমাদের অর্থনীতির চাকাকে সচল রাখবে। ডিজিটাল কমার্সকে শক্তিশালী করার জন্য তৈরিকৃত ‘বাংলাদেশ ডিজিটাল কমার্স পলিসি ২০১৮’ এর আওতায় বাণিজ্য মন্ত্রণালয় ই-কমার্স প্রতিষ্ঠানসমূহকে নীতিসহায়তা প্রদান অব্যাহত রেখেছে।বাণিজ্যমন্ত্রণালয় বলছে, বর্তমান পরিস্থিতিতে ই-কমার্সের গুরুত্ব অপরিসীম। সামাজিক দূরত্ব বজায় রেখে সাধারন মানুষের নিকট ওষুধ ও অন্যান্য অত্যাবশ্যকীয় পণ্য পৌছে দেয়ার দায়িত্ব পালন করছে ই-কমার্স। বাণিজ্য মন্ত্রণালয়ের প্রত্যক্ষ উদ্যোগে ই-কমার্স অ্যাসোসিয়েশনের সহযোগিতায় এ বছর আম মৌসুমে অন-লাইন আম মেলার আয়োজন করা হয়েছিল এবং এই কর্মসূচি বহুল প্রশংসিত হয়েছিল। বর্তমানে ই-কমার্সের মাধ্যমে ভোক্তাসাধারণের মধ্যে ন্যায্যমূল্যে টিসিবির পেয়াজ বিক্রয় কার্যক্রম সফলভাবে চলমান রয়েছে। ভবিষ্যতে এর পরিধি বাড়ানোর পরিকল্পনা ইতোমধ্যে গ্রহণ করা হয়েছে।')

In [32]:
from transformers import *
import tensorflow as tf
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = TFAutoModel.from_pretrained('/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor')
input_ids = tf.constant(tokenizer.encode("আমরা বাংলায় কথা বলি"))[None, :]  
outputs = model(input_ids)
last_hidden_states = outputs[0] 
print(last_hidden_states.numpy().shape)  

(1, 7, 768)


In [35]:
last_hidden_states

<tf.Tensor: shape=(1, 7, 768), dtype=float32, numpy=
array([[[-0.90135163, -0.36611202, -0.8293169 , ..., -1.3423055 ,
         -0.63330555,  0.7763298 ],
        [-0.90135163, -0.36611202, -0.8293169 , ..., -1.3423058 ,
         -0.63330555,  0.7763298 ],
        [-0.90135163, -0.36611205, -0.8293169 , ..., -1.3423057 ,
         -0.63330555,  0.7763298 ],
        ...,
        [-0.9013517 , -0.36611208, -0.82931703, ..., -1.3423059 ,
         -0.6333056 ,  0.7763299 ],
        [-0.9013517 , -0.36611208, -0.829317  , ..., -1.3423058 ,
         -0.63330555,  0.7763298 ],
        [-0.9013517 , -0.3661121 , -0.82931703, ..., -1.3423059 ,
         -0.63330567,  0.77632993]]], dtype=float32)>

In [34]:
outputs

(<tf.Tensor: shape=(1, 7, 768), dtype=float32, numpy=
 array([[[-0.90135163, -0.36611202, -0.8293169 , ..., -1.3423055 ,
          -0.63330555,  0.7763298 ],
         [-0.90135163, -0.36611202, -0.8293169 , ..., -1.3423058 ,
          -0.63330555,  0.7763298 ],
         [-0.90135163, -0.36611205, -0.8293169 , ..., -1.3423057 ,
          -0.63330555,  0.7763298 ],
         ...,
         [-0.9013517 , -0.36611208, -0.82931703, ..., -1.3423059 ,
          -0.6333056 ,  0.7763299 ],
         [-0.9013517 , -0.36611208, -0.829317  , ..., -1.3423058 ,
          -0.63330555,  0.7763298 ],
         [-0.9013517 , -0.3661121 , -0.82931703, ..., -1.3423059 ,
          -0.63330567,  0.77632993]]], dtype=float32)>,
 <tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[ 0.9995793 ,  0.9995571 , -0.99844664,  0.9998438 , -0.9962863 ,
          0.99873996, -0.99675786,  0.44712365, -0.9995195 , -0.9978436 ,
         -0.99935085, -0.9998417 , -0.99845624, -0.9995861 ,  0.99994373,
          0.9999